In [20]:
import pandas as pd

import folium

import geopandas as gp

import numpy as np

from scipy.spatial import ConvexHull

import random

from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt

import networkx as nx

import math

import h3



In [21]:
'''User defined functions'''
def visualize_grids(grids,folium_map):
    for grid in grids:
        boundary=h3.h3_to_geo_boundary(grid)
        boundary=[x for x in boundary]
        boundary.append(boundary[0])
        grid_PolyLine=folium.PolyLine(locations=boundary,weight=4,color='red')
        folium_map.add_child(grid_PolyLine) 
    return folium_map

'''Get Neighbor ranges'''

def Get_neighbors(y):
    x=list()
    for y_ in y:
        for y__ in y_:
            x.append(y__)
    return x
    

# visualize

In [22]:
City_data = gp.read_file('./Data/Guangdong/Guangdong.shp')

City_data = City_data.loc[(City_data['CityNameC']=='深圳市')&(City_data['SHI_D_']==4062)]


In [23]:
for idx,row in City_data.iterrows():
    
    Geo={'Shenzhen':row['geometry']}
    
np.save('./Data/Initialization/Geo',Geo)


In [24]:
'''Background'''

background = folium.Map([22.648883, 114.045011],tiles='CartoDB positron',zoom_start=12)

'''Zones'''

folium.Choropleth(
    geo_data=City_data,
    fill_color='blue',
    fill_opacity=0.0,
    name='Zone').add_to(background)


folium.LayerControl().add_to(background)


In [25]:
lat,lng=22.569494, 114.059183

resolution=8

grid_radius=0.461354684

h3_address=h3.geo_to_h3(lat, lng, resolution)

'''Neighbor'''

Neighbors=list(h3.hex_ring(h3_address, k=1))

'''Neighbor range'''

Neighbors=Get_neighbors(h3.hex_range_distances(h3_address, 2))

Neighbors



['88411cab1bfffff',
 '88411caa25fffff',
 '88411cab11fffff',
 '88411cab19fffff',
 '88411caa27fffff',
 '88411cab13fffff',
 '88411ca84dfffff',
 '88411ca869fffff',
 '88411cab17fffff',
 '88411ca845fffff',
 '88411cab53fffff',
 '88411ca841fffff',
 '88411caa21fffff',
 '88411cab1dfffff',
 '88411caa2dfffff',
 '88411cab57fffff',
 '88411caa23fffff',
 '88411cab15fffff',
 '88411ca849fffff']

# transform data

In [26]:
Geometry=np.load('./Data/Initialization/Geo.npy',allow_pickle=True).item()

boundary=[[[x[1],x[0]] for x in list(Geometry['Shenzhen'].exterior.coords)]]

geoJson = {'type': 'Polygon','coordinates': boundary}

Grids = list(h3.polyfill(geoJson, resolution))

np.save('./Data/Processed/Grids',Grids)

background=visualize_grids(Grids,background)

background=background.save('Example.html')